In [2]:
#import packages
import pandas as pd

In [3]:
#read data
df = pd.read_csv(r'C:\Users\emily\OneDrive\Desktop\CSE 6242\Project\dataset.csv')

In [4]:
df.head(3)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.1430,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.166,...,-17.235,1,0.0763,0.9240,0.000006,0.101,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,-9.734,1,0.0557,0.2100,0.000000,0.117,0.120,76.332,4,acoustic


In [131]:
def move_column(df, col_name, col_index):
    cols = df.columns.tolist()
    cols.insert(col_index, cols.pop(cols.index(col_name))) 
    df = df[cols]    
    return df

In [135]:
def make_categorical_column(df, col_name, col_index):
    bins = [0, .25, .50, .75, 100] 
    labels = ['Low', 'Moderate', 'High', 'Very High']
    df[col_name + '_categorical'] = pd.cut(df[col_name], bins=bins, labels=labels)
    df = move_column(df, col_name + '_categorical', col_index)
    df = df.rename(columns={col_name: col_name + '_numeric'})
    return df

In [139]:
def clean_project_data(df):
    """
    Function defined using fields from the Kaggle dataset. 
    https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset?resource=download
    """
    
#     remove index
    df = df.drop(df.columns[0], axis=1)
    
#    "artists"
    artist_cols = df['artists'].str.split(';', expand=True) #separate artists into individual columns
    artist_cols = artist_cols.iloc[:, :3].astype(str) #keep only first 3 artists
    artist_cols.columns = ["artist_1", "artist_2", "artist_3"] #name the new columns
    df = pd.concat([artist_cols, df], axis=1).drop(columns=['artists']) #replace original "artists" column in df
    
#     "album_name" and "track_name": convert to string
    df['album_name'] = df['album_name'].astype(str)
    df['track_name'] = df['track_name'].astype(str)
    
#     "popularity": leave original column but add a new categorical column
# note: popularity ranges from 0-100 with 0 being not popular and 100 being popular
    df['popularity'] = df['popularity'].astype(int) #convert to int
    df['popularity'] = df['popularity'] / 100 #convert to 0-1 scale to be consistent with other fields
    df = make_categorical_column(df, 'popularity', 6)

    
#     "duration_ms": convert to minutes
    df['duration_minutes'] = df['duration_ms'] / 60000
    df['duration_minutes'] = df['duration_minutes'].round(2)
    df = move_column(df, 'duration_minutes', 8)
    df = df.drop(columns=['duration_ms'])
    
#     "danceability"
# note: danceability ranges from 0-1 with 0 being not danceable and 1 being danceable
    df = make_categorical_column(df, 'danceability', 10)
    
#     "energy"
# ranges from 0-1
    df = make_categorical_column(df, 'energy', 12)
    
################# Consider dropping either energy or danceability? They seem similar
    
#     key
    #unsure of relevance at this time
    df = df.drop(columns=['key'])
    
#     loudness
    #unsure of relevance at this time
    df = df.drop(columns=['loudness'])
    
#     mode
    #unsure of what this field is, need more info
    
#     speechiness
# range 0-1, convert to 100
    df = make_categorical_column(df, 'speechiness', 15)
    
    
    return df    

In [140]:
clean_project_data(df).iloc[:3, 5:]

,popularity_numeric,popularity_categorical,duration_minutes,explicit,danceability_numeric,danceability_categorical,energy_numeric,energy_categorical,mode,speechiness_numeric,speechiness_categorical,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0.73,High,3.84,False,0.676,High,0.461,Moderate,0,0.1430,Low,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
1,0.55,High,2.49,False,0.420,Moderate,0.166,Low,1,0.0763,Low,0.9240,0.000006,0.101,0.267,77.489,4,acoustic
2,0.57,High,3.51,False,0.438,Moderate,0.359,Moderate,1,0.0557,Low,0.2100,0.000000,0.117,0.120,76.332,4,acoustic


In [126]:
df['speechiness'].max()

0.965